抓取座位數

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# 讀取你的航班代碼檔案
file_path = "flights.csv"  # 替換成你的檔案
data = pd.read_csv(file_path)

# 設定 Selenium 瀏覽器
driver_path = "/opt/homebrew/bin/chromedriver"  # 替換成你的 ChromeDriver 路徑
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path=driver_path, options=options)

# 定義爬取座位數的函數
def get_seat_count(flight_code):
    try:
        url = "https://flightera.net/en/search"
        driver.get(url)
        
        # 等待並找到搜尋欄位
        wait = WebDriverWait(driver, 10)
        search_box = wait.until(EC.presence_of_element_located((By.XPATH, '//input[@placeholder="Example : LH500"]')))
        
        # 輸入航班代碼並搜尋
        search_box.clear()
        search_box.send_keys(flight_code)
        search_box.send_keys(Keys.RETURN)
        
        # 等待結果頁加載
        time.sleep(3)  # 可改為更精確的條件等待
        
        # 抓取座位數資訊（假設座位數顯示在特定位置）
        seat_info = driver.find_element(By.XPATH, '//*[contains(text(), "Seats")]').text
        seat_count = int(''.join(filter(str.isdigit, seat_info)))  # 提取數字部分
        return seat_count
    except Exception as e:
        print(f"航班 {flight_code} 查詢失敗: {e}")
        return None

# 為每個航班代碼爬取座位數
data['seat_count'] = data['flight_code'].apply(get_seat_count)

# 關閉瀏覽器
driver.quit()

# 將結果保存至新檔案
output_file_path = "flights_with_seat_counts.csv"
data.to_csv(output_file_path, index=False)
print(f"已保存結果至：{output_file_path}")
